In [5]:
from pyspark.sql import SparkSession

In [6]:
spark=SparkSession.builder.appName("Test_3").getOrCreate()

In [7]:
spark

In [8]:
# Read csv file in local
df = spark.read.csv("C:\\Users\\IA-User\\Desktop\\diabetes.csv", header=True, inferSchema=True)

In [9]:
# Read all
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|


In [10]:
# Read 5 columns and rows
df.show(5)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows



In [11]:
# show schema
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [14]:
# Selecting particular column
df.select("Glucose", "Outcome").show(5)

+-------+-------+
|Glucose|Outcome|
+-------+-------+
|    148|      1|
|     85|      0|
|    183|      1|
|     89|      0|
|    137|      1|
+-------+-------+
only showing top 5 rows



In [22]:
df.select("Glucose", "Age").show()

+-------+---+
|Glucose|Age|
+-------+---+
|    148| 50|
|     85| 31|
|    183| 32|
|     89| 21|
|    137| 33|
|    116| 30|
|     78| 26|
|    115| 29|
|    197| 53|
|    125| 54|
|    110| 30|
|    168| 34|
|    139| 57|
|    189| 59|
|    166| 51|
|    100| 32|
|    118| 31|
|    107| 31|
|    103| 33|
|    115| 32|
+-------+---+
only showing top 20 rows



# Practice in SQL

In [25]:
# create a temporary view so that it can be used as a SQL table
df.createTempView("diabetes")

### 1. How many persons that are greater than 50 years old and has 2 or more child have diabetes?

In [26]:
df = spark.sql("SELECT SUM(Outcome) As Diabetesfifty From diabetes Where Pregnancies > 2 AND Age > 50").show()

+-------------+
|Diabetesfifty|
+-------------+
|           30|
+-------------+



### 2. Get total count of persons that have and do not have diabetes

In [30]:
df = spark.sql("""SELECT(SELECT COUNT(Outcome) FROM diabetes WHERE Outcome = 1) as people_with_diabetes,
               (SELECT COUNT(Outcome)
                FROM diabetes WHERE Outcome = 0) as no_people_with_diabetes""").show()

+--------------------+-----------------------+
|people_with_diabetes|no_people_with_diabetes|
+--------------------+-----------------------+
|                 268|                    500|
+--------------------+-----------------------+



### 3. Get total count of persons that have diabetes per age

In [29]:
df = spark.sql("""SELECT SUM(CASE WHEN Age < 18 THEN 1 ELSE 0 END) AS Under_18,
               SUM(CASE WHEN Age BETWEEN 18 AND 24 THEN 1 ELSE 0 END) AS BETWEEN_18_24,
               SUM(CASE WHEN Age BETWEEN 25 AND 34 THEN 1 ELSE 0 END) AS BETWEEN_25_34,
               SUM(CASE WHEN Age BETWEEN 35 AND 44 THEN 1 ELSE 0 END) AS BETWEEN_35_44,
               SUM(CASE WHEN Age BETWEEN 45 AND 54 THEN 1 ELSE 0 END) AS BETWEEN_45_54,
               SUM(CASE WHEN Age BETWEEN 55 AND 64 THEN 1 ELSE 0 END) AS BETWEEN_55_64,
               SUM(CASE WHEN Age BETWEEN 65 AND 74 THEN 1 ELSE 0 END) AS BETWEEN_65_74,
               SUM(CASE WHEN Age BETWEEN 75 AND 84 THEN 1 ELSE 0 END) AS BETWEEN_75_84,
               SUM(CASE WHEN Age BETWEEN 85 AND 94 THEN 1 ELSE 0 END) AS BETWEEN_85_94
            FROM diabetes """).show()

+--------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|Under_18|BETWEEN_18_24|BETWEEN_25_34|BETWEEN_35_44|BETWEEN_45_54|BETWEEN_55_64|BETWEEN_65_74|BETWEEN_75_84|BETWEEN_85_94|
+--------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|       0|          219|          269|          147|           79|           38|           15|            1|            0|
+--------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+

